# Déclaration

In [215]:
from importlib import reload
import numpy as np
import time

import pyautogui
import pandas as pd
import re
import keyboard
import os
import fct
reload(fct)

wave=pd.read_csv('wave.csv')

class Perso():
    """_summary_
    num_perso numéro du perso à séléctionner
    raid faire attention au horaires des maps
    
    """
    def __init__(self,num_perso=2,blue_map=True,raid=False,region_map=False):
        self.filtre_actif= region_map
        self.num_perso=num_perso
        self.test_in_game()
        self.region_map=region_map
        self.blue_map=blue_map
        self.actif_raid=raid

    def test_in_game(self):
        path1=f'screen/launch/test_in_game.png'
        if pyautogui.locateOnScreen(path1, grayscale=False, confidence=0.95) is None :
            if pyautogui.locateOnScreen(f'screen/launch/in_tower.png',grayscale=True,confidence=0.9) is None :
                if fct.launch(self.num_perso) is False:
                        fct.close_game("test_in_game")
            
    def launch_map(self):
        if self.region_map:
            #sélection par region
            np.random.shuffle(list_map)
            for m in list_map:
                find=fct.find_region(m)
                if find:
                    pyautogui.rightClick( find)
                    break               
        else:
            #sélection par couleur
            if not self.filtre_actif:
                fct.filtre_map(list_map)
                self.filtre_actif=True
            for i in range(6):
                pyautogui.moveTo(2400,1050)
                for i in range(0,15): pyautogui.scroll(-3000)
                pyautogui.moveTo(1300,700)
                time.sleep(0.2)
                for m in list_map:
                    find = pyautogui.locateOnScreen(f'screen/map/{m}.png',region=[1930,363,2445,1080], confidence=0.90)
                    if find:
                        pyautogui.rightClick( pyautogui.center(find))
                        break
                    if self.blue_map:  
                        find=fct.find_blue(f'screen/map/{m}_b.png')
                        if find: 
                            pyautogui.rightClick( pyautogui.center(find))
                            break
                if find: break

        time.sleep(1)
        fct.click_icone('screen/launch/activate.png',3,0.3,False,0.8)
        time.sleep(1)
        if fct.click_icone('screen/launch/croix.png',1):
            time.sleep(1)
            return False
        if fct.go_map(wave)==False:return False
        fct.click_icone('screen/launch/waypoint.png',1)#à cause ges map jaunes
        return m
    
    def run_map(self,m,wave,retry=True,ancien_run=1):
        for i in range(10):
            if fct.in_map():break
            time.sleep(1)
        fct.click_icone('screen/launch/tab_map.png',1,confidence=0.85)
        pyautogui.moveTo(700,700)
        time.sleep(0.2)
        name=m
        for i in range(10):
            if pyautogui.locateOnScreen(f'screen/map/{m}_1.png',confidence=0.85-(i*0.01)):
                name+= '_1'
                break
            elif pyautogui.locateOnScreen(f'screen/map/{m}_2.png',confidence=0.85-(i*0.01)):
                name= m + '_2'
                break
            elif i==9:
                print("erreur début de map")
                name=fct.error_start_map(list_map)
                if name==False:
                    fct.leave_map()
                    return False
                print(f"map récupére {name}")
                break
            time.sleep(1)
        end=False
        cpt=0
        fct.prepar_run(self.num_perso)
        #run
        start_gold=time.perf_counter()
        for i,n in enumerate(wave[name]):
            if pd.isna(n) :
                break
            x,y=re.findall(r'\d+',n)
            x=int(x)
            y=int(y)
            #perso2
            #if self.num_perso==1:time.sleep(1)
            if i%5==0 and self.num_perso>=2:
                fct.keyboard('q',0.9)
            
            if not fct.clik_move_map(x,y):
                fct.clik_move_map(x,y)
                
            #pause entre deux points
            start_time = time.perf_counter()
            time_wait=fct.time_run(x,y)*1.3 if self.num_perso==1 else fct.time_run(x,y)
            while time.perf_counter()-start_time < time_wait:
                #test pause utilisateur    
                if pyautogui.position()[0]< 0:
                    print(i,end=" ")
                    time.sleep(0.9)
                    while not keyboard.is_pressed('+'):
                        time.sleep(0.05)
                    fct.keyboard('q',0.2)
                    print("end_pause")
                    pyautogui.doubleClick(1200,1400)
                    start_gold=time.perf_counter()
                    if self.num_perso ==2:
                        fct.keyboard('q',0.7)
                                 
                #test ennemi found
                if pyautogui.locateOnScreen(f'screen/map/ennemi.png',region=[1850,920,1900,1210],confidence=0.7,grayscale=True):
                    start_gold=time.perf_counter()
                if time.perf_counter()-start_gold>50 and (retry==True or ancien_run==2): 
                    print("no gold found")
                    end=True
                    break
            if end:break
            if fct.death():return False 
               
        if not fct.kill_boss(wave[name[:-2]+'_boss'],retry,self.num_perso):
            if retry:
                fct.escape()
                self.run_map(name[:-2],wave,False,int(name[-1]))

## Enregistrement

In [44]:
pyautogui.leftClick(1200,1400)
time.sleep(0.5)
pyautogui.leftClick(1200,1400)

while 1:
    """    while not keyboard.is_pressed('space'):
        time.sleep(0.1)              
        """
    time.sleep(2.7)
    x,y=pyautogui.position()
    print(f'{x},{y}')        
     
     
    if 1:
        time.sleep(0.2)
        fct.keyboard('q',0.7)
        time.sleep(1)
       
    if not fct.clik_move_map(x,y): 
        fct.clik_move_map(x,y)

    


1042,692
1189,670
1390,1132
1486,1144
1588,1120
1858,1163
2006,996
1998,448
1564,169
1351,163
1737,199
1937,253
2297,664
2343,737
2385,818
2160,534
2116,450
1600,159
1483,175
1314,171
1077,140
1672,219
1782,248
1929,428
1844,304
1844,304
-1108,650
2147,523
2146,523
1944,1098
1944,1098
1183,1192
1649,1258
1628,1221
1628,1221
1652,1212
1565,1210
1506,1210
1678,1202
1927,969
2078,887
2220,883
2261,800
2112,567
1563,231
1206,231
858,204
709,253
709,253
-894,512
-1189,317


KeyboardInterrupt: 

## Parcours un colonne

In [21]:
cpt=0
pyautogui.leftClick(1200,1400)
time.sleep(0.5)
pyautogui.leftClick(1200,1400)

while 1:
    time.sleep(2.5)
    wave=pd.read_csv('wave.csv')
    n=wave.loc[cpt,"Lighting_1"]
    if pd.isna(n) == False:
        n=re.findall(r'\d+',n)
        if cpt%5==0:
            time.sleep(0.2)
            fct.keyboard('q',0.5)
            time.sleep(1)
            
           
        if not fct.clik_move_map(int(n[0]),int(n[1])):
            print(1)
            if not fct.clik_move_map(int(n[0]),int(n[1])):
                print(2)
                
                fct.clik_move_map(int(n[0]),int(n[1]))
       
        print(cpt,end=' ')
        cpt+=1

1
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 1
32 33 34 35 36 37 1
38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 

KeyboardInterrupt: 

In [134]:
import fct
reload(fct)
time.sleep(2)
m='Plague'

perso=Perso(1,raid=False)
perso.run_map(m,wave)


KeyboardInterrupt: 

In [135]:
reload(fct)
fct.raid([18.45,20])
list_map
m

'Plague'

In [140]:
#Team Viewer
pyautogui.leftClick(2520,1120)
time.sleep(5)
pyautogui.leftClick(1536,792)

# Main

In [234]:
reload(fct)
list_map=["Lighting","Air","Brazier","Greed" ,"Cycle","Wanderer","Plague","Shepherd","Flash"]#,,
      
perso=Perso(num_perso=2,raid=False)
if perso.num_perso==1:
    perso.region_map=True
    
while 1:
    if not(fct.raid([15.35,19.15],[7.45,11.10]) and perso.actif_raid) \
    and fct.launch_tower(wave,perso.num_perso):
        m=perso.launch_map()
        if m: 
            perso.run_map(m,wave)
            fct.se_vider(sell=True)
            fct.alchimie(wave,perso.num_perso)
    else:
        fct.close_game()
        for i in range(60*5): time.sleep(1)

Looking For launch/reset.png
Looking For launch/start_filtre.png
Looking For launch/select_all.png
Looking For map/ok_filtre.png
Looking For launch/croix.png
Looking For launch/reset.png
Looking For launch/waypoint.png
Looking For launch/reset.png
Looking For launch/ok.png
Looking For launch/reset.png
Looking For launch/storage.png
Looking For launch/reset.png
Looking For alchimie/atelier.png
Looking For alchimie/atelier_vide.png
Looking For launch/reset.png
Looking For launch/croix.png
Looking For launch/reset.png
Looking For launch/waypoint.png
Looking For launch/tab_map.png
Looking For launch/reset.png
Looking For launch/ok.png
Looking For launch/reset.png
Looking For launch/storage.png
Looking For launch/reset.png
Looking For alchimie/atelier.png
Looking For alchimie/atelier_vide.png
Looking For launch/reset.png
Looking For launch/croix.png
Looking For launch/reset.png
Looking For launch/waypoint.png
Looking For launch/tab_map.png
no gold found
Looking For launch/boss.png
Looking F

KeyboardInterrupt: 

## test

In [163]:
reload(fct)
time.sleep(1)
#fct.close_game("no go tower",perso.num_perso)
fct.raid([16.45,18.45],[9.45,11.15])
m="Brazier"
fct.find_region(m)


Looking For launch/select_all.png
Looking For map/ok_filtre.png


False

In [169]:
np.random.shuffle(list_map)

In [170]:
list_map

['Greed',
 'Wanderer',
 'Lighting',
 'Plague',
 'Brazier',
 'Cycle',
 'Flash',
 'Air',
 'Shepherd']

In [65]:
i=0
start_time = time.perf_counter()
while time.perf_counter()-start_time < fct.time_run(200,200):
    find = pyautogui.locateOnScreen(f'screen/map/ennemi.png', confidence=0.8,grayscale=True)
    if find:
        print(1,end='')
    else:
        print(" ",end="")
    i+=1
    if i ==150: 
        print(i)
        i=0

In [58]:

i=0 
for j in range(int(27/1.54)):
    if pyautogui.position()[0]< -11100:
        print(i,end=" ")
        time.sleep(0.9)
    find = pyautogui.locateOnScreen(f'screen/map/ennemi.png', confidence=0.8,grayscale=True)
    if find:
        print(1,end='')
    else:
        print(" ",end="")
    i+=1
    if i ==150: 
        print(i)
        i=0 
# Get the end time
end_time = time.perf_counter()



# Print the elapsed time
print(f'Code execution took {elapsed_time:.6f} seconds')

                 Code execution took 2.658209 seconds


In [355]:

    
    
time_run(1400,1250)

148

In [298]:
time.sleep(1)
pyautogui.moveTo(2400,1050)
for i in range(0,12):
    pyautogui.scroll(-3000)


In [113]:
for m in list_map:
        find = pyautogui.locateOnScreen(f'screen/map/{m}.png', confidence=0.98)
        if find:
            pyautogui.rightClick( pyautogui.center(find))
            print(pyautogui.screenshot().getpixel((2171+25,394+53)))
        elif 1:  
            find=fct.find_blue(f'screen/map/{m}_b.png')
            if find: 
                pyautogui.rightClick( pyautogui.center(find))
                print(pyautogui.screenshot().getpixel((2171+25,394+53)))
            #quest journa
            if os.path.exists(f'screen/map/{m}_bq.png'):
                find=fct.find_blue(f'screen/map/{m}_bq.png')
                if find: 
                    pyautogui.rightClick( pyautogui.center(find))
                    print(pyautogui.screenshot().getpixel((2171+25,394+53)))

In [354]:
4.3


1

In [44]:
if pyautogui.locateOnScreen(f'screen/launch/ok_map.png',confidence=0.7):print(1)

1


In [115]:
print(pyautogui.screenshot().getpixel((i[0]+i[2]-5,i[1]+i[3]-5)))
print(pyautogui.screenshot().getpixel((2069+25,394+53)))
print(pyautogui.screenshot().getpixel((2171+25,394+53)))


(25, 26, 41)
(47, 40, 19)
(54, 37, 16)


In [116]:
find = pyautogui.locateAllOnScreen(f'screen/map/Plague_b.png', confidence=0.99)
for i in find:
    print(i)
    print(pyautogui.screenshot().getpixel((i[0]+i[2]-5,i[1]+i[3]-5)))
    

Box(left=1960, top=493, width=63, height=38)
(22, 22, 33)
Box(left=2062, top=493, width=63, height=38)
(26, 27, 40)
Box(left=2163, top=493, width=63, height=38)
(25, 25, 36)
Box(left=2265, top=493, width=63, height=38)
(24, 24, 35)
Box(left=2366, top=493, width=63, height=38)
(28, 28, 41)
Box(left=1960, top=593, width=63, height=38)
(21, 22, 35)
Box(left=2062, top=593, width=63, height=38)
(28, 28, 45)
Box(left=2163, top=593, width=63, height=38)
(26, 26, 37)
Box(left=2265, top=593, width=63, height=38)
(25, 25, 38)
Box(left=2366, top=593, width=63, height=38)
(27, 28, 43)
Box(left=1960, top=694, width=63, height=38)
(25, 26, 41)


## test dps

In [2]:
# Initialisation de la variable qui contiendra la suite
suite = [1]

# Initialisation de la variable qui contiendra la somme
somme = 1

# Boucle pour générer les éléments de la suite
for i in range(1,6):
    # Calcul de l'élément suivant en décrémentant de 15%
    element = suite[i-1] * (0.7)
    # Ajout de l'élément à la suite
    suite.append(element)
    # Ajout de l'élément à la somme
    somme += element

# Affichage de la suite
print(suite)

# Affichage de la somme
print("La somme des éléments de la suite est :", somme*0.8)


[1, 0.7, 0.48999999999999994, 0.3429999999999999, 0.24009999999999992, 0.16806999999999994]
La somme des éléments de la suite est : 2.3529359999999997


In [213]:
2/1.36

1.4705882352941175

In [6]:
2.35/(1.2*1.3*1.3)


1.1587771203155819

## ACTE 10

In [283]:
def run_acte(wave,retry=True,run_time=0):
    if retry:run_time=time.perf_counter()
    end=False
    fct.prepar_run(2)
    #run
    
    for i,n in enumerate(wave["Air_1"]):
        if pd.isna(n) :
            break
        x,y=re.findall(r'\d+',n)
        x=int(x)
        y=int(y)
        #perso2
        if i%5==0 :
            fct.keyboard('q',0.9)
        
        if not fct.clik_move_map(x,y):
            fct.clik_move_map(x,y)
            
        #pause entre deux points
        start_time = time.perf_counter()
        time_wait=fct.time_run(x,y) #*1.2
        while time.perf_counter()-start_time < time_wait:
            #test pause utilisateur    
            if pyautogui.position()[0]< 0:
                print(i,end=" ")
                time.sleep(0.9)
                while not keyboard.is_pressed('+'):
                    time.sleep(0.05)
                fct.keyboard('q',0.2)
                print("end_pause")
                pyautogui.doubleClick(1200,1400)
                fct.keyboard('q',0.7)
                                
            #test ennemi found
            if pyautogui.locateOnScreen(f'screen/map/ennemi.png',region=[1850,920,1900,1210],confidence=0.7,grayscale=True):
                start_gold=time.perf_counter()
        if fct.death():
            time.sleep(26)
            return False 
    
    fct.escape()
    time.sleep(4)
    pyautogui.leftClick(900,626,duration=0.3)
    fct.keyboard('f')
    pyautogui.leftClick(1150,626,duration=0.3)
    time.sleep(7)
    if fct.in_city():
        return True 
    
        
    time.sleep(4)
    fct.leave_map()
    time.sleep(2.5)
    return True

In [312]:
#ACTE10
#perso=Perso(3,False)
reload(fct)
#cpt_no_go_map=0
while 1:
    fct.go_waypoint()
    fct.take_waypoint()
    pyautogui.leftClick(1053,1005,duration=0.3)
    pyautogui.leftClick(1900,900,duration=0.3)
    fct.click_icone('screen/launch/ok.png')
    fct.click_icone('screen/launch/reset.png')
    time.sleep(1.5)
    pyautogui.leftClick(1300,1400,duration=0.2)
    pyautogui.leftClick(1020,1200,duration=0.3)
    time.sleep(1.5)
    pyautogui.leftClick(1020,1200,duration=0.3)
    time.sleep(1.5)
    pyautogui.leftClick(750,727,duration=0.3)
    time.sleep(1.5)
    #Acte4
    pyautogui.leftClick(450,927,duration=0.3)
    #Acte10
    #pyautogui.leftClick(600,900,duration=0.3)
    fct.keyboard('f')
    time.sleep(5)
    #cpt_no_go_map+=1
    #if cpt_no_go_map==10: break
    
    if fct.in_map():
        #cpt_no_go_map=0
        
        run_acte(wave)
        #se vider
        if fct.in_city():
            if fct.go_blacksmith()==False:
                fct.go_blacksmith()
        time.sleep(0.5)
        if fct.go_waypoint() :
            fct.click_icone('screen/launch/icone_storage_acte10.png')
            time.sleep(3)
            fct.click_icone('screen/launch/storage.png',1)
            fct.click_icone('screen/launch/storage_clear.png',1)
            fct.click_icone('screen/launch/croix.png')
            time.sleep(0.5)    
        for i in range(np.random.randint(30, 120)):time.sleep(1)
        
        

    

Looking For launch/reset.png
Looking For launch/ok.png
60 end_pause
Looking For launch/reset.png
Looking For launch/normal.png
Looking For launch/rare.png
Looking For launch/normal.png
Looking For launch/rare.png
Looking For launch/ok.png
Looking For launch/croix.png
Looking For launch/reset.png
Looking For launch/storage.png
Looking For launch/reset.png
Looking For launch/reset.png
Looking For launch/normal.png
Looking For launch/magic.png
Looking For launch/rare.png
Looking For launch/normal.png
Looking For launch/magic.png
Looking For launch/rare.png
Looking For launch/ok.png
Looking For launch/croix.png
Looking For launch/reset.png
Looking For launch/storage.png
Looking For launch/reset.png
Looking For launch/reset.png
Looking For launch/normal.png
Looking For launch/magic.png
Looking For launch/rare.png
Looking For launch/normal.png
Looking For launch/magic.png
Looking For launch/rare.png
Looking For launch/ok.png
Looking For launch/reset.png
Looking For launch/storage.png
Looking

KeyboardInterrupt: 